In [ ]:
from comet_ml import Experiment
import torch
import utils, data_setup, engine, model_builder

In [ ]:
# Create global variables
PATH = "/kaggle/input/dog-breed-classifiation"
MODEL_CHOICE = "resnet50"

parameter = {
    "batch_size": 32,
    "epochs" : 5,
    "learning_rate": 0.001,
    "img_size" : 224
}

In [ ]:
def run_experiment(model_choice, path, parameter):
    # Init Comet
    experiment = Experiment(
        api_key= "5Cd7hajJ3PuFPv85lVNcmB2lm",  # Insert your api key
        project_name= "dog-breed-classification", # Insert your project name
        workspace= "bernandogunawan" # Insert your workspace
    )

    # Device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Log parameters
    experiment.log_parameters(parameter)
    experiment.log_parameter("device", device)

    # Model
    model,weight = model_builder.build_model(
        model_choice = model_choice,
        num_classes = 120
    )
    model = model.to(device)

    experiment.set_name(f"{model_choice} with {parameter['epochs']} epochs and {parameter['learning_rate']} learning rate")

    # preprocess
    train_dataloader,test_dataloader,class_name = data_setup.create_dataloader(
        path = path,
        transform=weight,
        image_size = parameter['img_size'],
        batch_size = parameter['batch_size']
    )

    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=parameter['learning_rate']
    )

    for epoch in range(parameter['epochs']):
            train_loss, train_acc = engine.train_step(
                model, train_dataloader, optimizer, loss_func)
            test_loss, test_acc = engine.evaluate(
                model, test_dataloader, loss_func)
            
            print(f"Epoch: {epoch+1} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {test_loss:.4f} | val_acc: {test_acc:.4f}")


            experiment.log_metrics({
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": test_loss,
                "val_acc": test_acc
            }, step=epoch)

    experiment.end()
    return model

In [ ]:
model_list = ["efficientnet_b2", "resnet50"]
epochs = [5, 10]
learning_rate = [0.001, 0.0001]

for model_choice in model_list:
    for epoch in epochs:
        for lr in learning_rate:
            model = run_experiment(MODEL_CHOICE, PATH, parameter)
            utils.save_model(
                model = model,
                model_name = f"{model_choice}_{epoch}_{lr}.pth"
            )

In [ ]:
# save model
utils.save_model(
    model = model,
    model_name = f"{MODEL_}_freeze.pth"
)